# What is VNC?

VNC is a remote desktop and screen sharing service.

In [ ]:
fluxbox -display :0
x11vnc -forever -nopw -shared -rfbport -ncache 5900 -display :0



Finally, run this middle-ware for the html5 client


In [ ]:
../novnc/utils/launch.sh --vnc localhost:5900


VNC HTML client?

* https://github.com/novnc/noVNC.git


In [17]:
mkdir -p novnc ; if git --work-tree=./novnc branch | grep 'master'; then \
echo "Already checked out novnc"; \
else \
git clone https://github.com/novnc/noVNC.git ./novnc ; \
fi ; ls -la novnc ; pwd


Use vnc with live-server?

In [15]:
npm i -g live-server
live-server novnc


Use VNC in Docker?

Selenium Docker configured x11vnx and Xvfb for us!


In [ ]:

RUN dos2unix /home/seluser/novnc/utils/launch.sh
RUN sed -i -e 's/export GEOMETRY.*/export GEOMETRY="8160x6120x24"/g' /opt/bin/entry_point.sh
RUN sed -i -e 's/x11vnc/x11vnc -noxdamage/g' /opt/bin/entry_point.sh
RUN sed -i '/wait \$NODE_PID/i/home/seluser/novnc/utils/launch.sh --vnc localhost:5900 &' /opt/bin/entry_point.sh
USER seluser
